In [ ]:
!pip install pandas
!pip install numpy
!pip install ffn
!pip yfinance

In [136]:
import pandas as pd 
import numpy as np 
import ffn
#取樣週期，MS = 月 , W=周 , D =日
sample = 'MS'
#參考大盤指數，^GSPC,^Twii,^DJI,^IXIC
index_name = "^twii"
#資產輸入，需要至yahoo finance查詢代碼 (台灣上市+.TW，上櫃加.TWO)
stock_list = ["2454.TW","2330.TW"]
#開始時間
start_time = "2016-01-01"


# 下方不需變動

In [137]:
#名稱前處理
stock_high_list = ""
stock_low_list = ""
stock_close = ""

for stock in stock_list :
    stock_high = stock + ":High"
    stock_low  = stock + ":Low"
    stock_high_list = stock_high_list +","+ stock_high
    stock_low_list = stock_low_list +","+ stock_low
    stock_close = stock_close +","+ stock  
#抓取資產開高低收
close = ffn.get(stock_close[1:] +"," +index_name ,start = start_time) 
high = ffn.get(stock_high_list[1:] +"," +index_name + ":High",start = start_time) 
low = ffn.get(stock_low_list[1:] +"," +index_name+ ":Low",start = start_time) 
interest_rate = ffn.get("^tnx",start = start_time) 

#取樣週期調整利率參數
if sample.upper()  == 'MS' :
    r_div = 12
elif sample.upper() == "W" :
    r_div = 52
elif sample.upper() == "D" :
    r_div = 365
#column名稱前處理
non_index_stock_list = stock_list
index = index_name[1:]
stock_list.append(index)
#抽樣頻率調整資產
if sample.upper() == "W":
    Month_first = close.resample('W', label = 'left', closed = 'left').first()
    Month_max =  high.resample('W', label = 'left', closed = 'left').max()
    Month_low =  low.resample('W', label = 'left', closed = 'left').min()
else :
    Month_first = close.resample(sample).first()
    Month_max =  high.resample(sample).max()
    Month_low =  low.resample(sample).min()
Month_max.columns = stock_list
Month_low.columns = stock_list

if sample.upper() == "W"  : 
    r = interest_rate.resample('W', label = 'left', closed = 'left').mean() / 100 / r_div
else :
    r = interest_rate.resample(sample).mean() / 100 / r_div

#Month_max =  prices.groupby([prices.index.year,prices.index.month]).agg(max)
#print(Month_first)
#print(Month_max)
#print(Month_low)
#計算pi
pi = pd.DataFrame(np.nan, index=Month_first.index ,columns = ["pi"])
pi = (Month_first[index]*(1+r["tnx"]) - Month_low[index]) / (Month_max[index] - Month_low[index])

#各資產計算並儲存
stock_price = {} 
for stock in non_index_stock_list :
    stock_price[stock] = (pi * Month_max[stock]) + ((1-pi)*Month_low[stock]) /(1+r["tnx"])
stock_price = pd.DataFrame(stock_price).dropna()
stock_price.to_csv("stock_prices.csv")
print(stock_price)


               2454.TW     2330.TW          twii
Date                                            
2016-01-01  235.881598  139.749474   8124.526872
2016-02-01  213.103832  146.325103   8162.529332
2016-03-01  238.882517  152.118221   8489.297491
2016-04-01  241.375542  159.944316   8668.181798
2016-05-01  209.825355  151.064330   8301.089796
...                ...         ...           ...
2022-08-01  683.780354  507.464033  15000.625524
2022-09-01  650.691757  489.553857  14842.153445
2022-10-01  576.554050  414.977460  13326.242662
2022-11-01  600.686490  389.953588  13042.252939
2022-12-01  741.497247  502.049774  15051.063139

[84 rows x 3 columns]
